In [1]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from fake_headers import Headers
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# 엑셀 파일 경로
file_path = 'movie_list.xlsx'

# 엑셀 파일을 DataFrame으로 읽기
df = pd.read_excel(file_path)

def generate_payload(start_date, end_date):
    current_date = datetime.now().strftime("%Y-%m-%d")
    return {
        "CSRFToken": "G1P5qi2QQzdb48cAT4etN5FEw5HKBf3uH2jfE9rWzEI",
        "loadEnd": "0",
        "curTime": current_date,
        "totSeatCntRatioOrder": "",
        "totSeatCntOrder": "",
        "totShowAmtOrder": "",
        "addTotShowAmtOrder": "",
        "totShowCntOrder": "",
        "addTotShowCntOrder": "",
        "dmlMode": "search",
        "startDate": start_date,
        "endDate": end_date,
        "searchType": "",
        "repNationCd": "",
        "wideareaCd": ""
    }

def fetch_data(url, payload, target_movie, period):
    headers = Headers(headers=True).generate()
    
    session = requests.Session()
    retry = Retry(
        total=10,
        read=10,
        connect=10,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    time.sleep(1)  # 각 작업 시작 전에 1초 대기
    try:
        response = session.post(url, headers=headers, data=payload, timeout=20)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching data for {target_movie} during {period}: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    table_datas = soup.find('tbody', {'id': 'mainTbody'}).find_all('tr')
    results = []

    for row in table_datas:
        movie_name = row.find('td', {'id': 'td_movie'}).get_text(strip=True)
        if movie_name == target_movie:
            if 'findPeriodSeatTicketList' in url:
                ratio = row.find('td', {'id': 'td_totSeatCntRatio'}).get_text(strip=True)
            elif 'findPeriodShowTicketList' in url:
                ratio = row.find('td', {'id': 'td_totDspCntRatio'}).get_text(strip=True)
            elif 'findPeriodScreenTicketList' in url:
                ratio = row.find('td', {'id': 'td_totScrnCntRatio'}).get_text(strip=True)
            results.append((movie_name, ratio, period))
    
    return results

def get_movie_ratios(movie_name, release_date_str):
    release_date = datetime.strptime(release_date_str, "%Y-%m-%d")
    periods = [(0, 7), (7, 14), (14, 21)]

    urls = {
        "seat": "https://kobis.or.kr/kobis/business/stat/boxs/findPeriodSeatTicketList.do",
        "show": "https://kobis.or.kr/kobis/business/stat/boxs/findPeriodShowTicketList.do",
        "screen": "https://kobis.or.kr/kobis/business/stat/boxs/findPeriodScreenTicketList.do"
    }

    results = {
        "seat": [],
        "show": [],
        "screen": []
    }

    for start, end in periods:
        start_date = (release_date + timedelta(days=start)).strftime("%Y-%m-%d")
        end_date = (release_date + timedelta(days=end)).strftime("%Y-%m-%d")
        period = f"{start_date} ~ {end_date}"
        payload = generate_payload(start_date, end_date)

        results["seat"].extend(fetch_data(urls["seat"], payload, movie_name, period))
        results["show"].extend(fetch_data(urls["show"], payload, movie_name, period))
        results["screen"].extend(fetch_data(urls["screen"], payload, movie_name, period))

    return movie_name, results

# 결과를 저장할 DataFrame 생성
result_columns = ['영화명', '기간', '좌석점유율', '상영점유율', '스크린점유율']
result_df = pd.DataFrame(columns=result_columns)

# 각 영화별로 데이터 수집 및 저장
processed_movies = set()
tasks = []

# 테스트로 10개의 영화만 선택
test_df = df.drop_duplicates(subset=['영화명']).head(10)

with ThreadPoolExecutor(max_workers=10) as executor:
    for index, row in tqdm(df.iterrows(), total=len(df.drop_duplicates(subset=['영화명'])['영화명'])):
        movie_name = row['영화명']
        release_date_str = row['개봉일'].strftime("%Y-%m-%d")
        
        if movie_name not in processed_movies:
            processed_movies.add(movie_name)
            time.sleep(1)  # 각 프로세스 시작 전에 1초 대기
            tasks.append(executor.submit(get_movie_ratios, movie_name, release_date_str))
    
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        movie_name, ratios = future.result()
        
        seat_data = []
        for period in ratios["seat"]:
            seat_data.append({
                '영화명': period[0],
                '기간': period[2],
                '좌석점유율': period[1],
                '상영점유율': '',
                '스크린점유율': ''
            })
            
        for period in ratios["show"]:
            for item in seat_data:
                if item['영화명'] == period[0] and item['기간'] == period[2]:
                    item['상영점유율'] = period[1]
            
        for period in ratios["screen"]:
            for item in seat_data:
                if item['영화명'] == period[0] and item['기간'] == period[2]:
                    item['스크린점유율'] = period[1]
        
        result_df = pd.concat([result_df, pd.DataFrame(seat_data)], ignore_index=True)

# 결과를 새로운 엑셀 파일에 저장
output_file_path = 'movie_ratios.xlsx'
result_df.to_excel(output_file_path, index=False)

print(f"결과가 {output_file_path}에 저장되었습니다.")


100%|██████████| 10/10 [00:13<00:00,  1.30s/it]

결과가 movie_ratios.xlsx에 저장되었습니다.
